<a href="https://colab.research.google.com/github/ftvalentini/misc-notebooks/blob/master/borges_hipalages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buscando hipálages con NLP
## Una aplicación con los cuentos de J.L. Borges

**ACA VA UN RESUMEN**
(usar GPU???)

### Descargas y librerías

1. Instalamos [Stanza](https://github.com/stanfordnlp/stanza/), la librería desarrollada por Stanford que vamos a usar para encontrar las dependencias gramaticales en los textos. La librería ofrece modelos pre-entrenados para múltiples idiomas para aplicar en el contexto de tareas de NLP (en nuestro caso, dependency parsing). 

In [0]:
!pip install stanza

2. Cargamos los vectores [GloVe](https://nlp.stanford.edu/pubs/glove.pdf), word embeddings ajustados por Stanford. En particular, vamos a usar los vectores de mayor dimensión posible (300) y ajustados con la wikipedia en inglés. ¡Atención que este paso puede demorar hasta 20 minutos! (En Google Colab no debería tardar más de 5 minutos).

In [0]:
def load_embeddings():
    """ Load GloVe Vectors
        Returns: array of size vocab_size, embeddings_dim)
    """
    import gensim.downloader as api
    vectors = api.load("glove-wiki-gigaword-300")
    return vectors

In [0]:
### PUEDE TARDAR VARIOS MINUTOS ###
word_vectors = load_embeddings()

3. Cargamos las librerías que vamos a necesitar para hacer el análisis. También descargamos los modelos de tokenización `punkt` porque son necesarios para separar el texto en oraciones.

In [0]:
import stanza
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

4. Descargamos los modelos de `stanza` para textos en inglés. Los modelos se guardan por defecto en `/root/stanza_resources`. Este paso tarda no más de 5 minutos en Google Colab pero puede tardar más en otro entorno.

In [0]:
### PUEDE TARDAR VARIOS MINUTOS ###
stanza.download('en')

5. Levantamos el texto que vamos a analizar. En este caso, La Biblioteca de Babel, guardada en texto plano. Lo vamos a almacenar como una cadena de carácteres.

In [0]:
txt_file = 'the_library_of_babel.txt'
with open(txt_file, "r", encoding='utf-8') as f:
    texto = f.read() 

### Limpieza del texto

Limpiamos el texto siguiendo los siguientes pasos:  
a. Reemplazamos los saltos de linea y cualquier otro carácter de tipo whitespace por espacios no repetidos, porque no nos interesan los cambios de párrafo ni de página, sino solamente las oraciones  
b. Tokenizamos el texto en oraciones.  
c. Juntamos varias oraciones en batches separando cada oración con dos saltos de linea `\n\n` -- de acuerdo a la documentación de `stanza` el procesamiento de muchos documentos es más veloz de esta manera. 

In [0]:
def clean_texto(texto):
    """Cleans and tokenizes raw text
       Returns: list of clean sentences
       by \n\n 
    """
    clean_text = " ".join(texto.split())
    sentences = sent_tokenize(clean_text, language='english')
    return sentences

def create_batches(sentences, sentences_per_doc=16):
    """Creates batches of sentences
       Returns: list of strings, each one containg multiple sentences separated
       by \n\n 
    """
    ranges = [(max(0,i), min(i+sentences_per_doc,len(sentences))) \
                 for i, x in enumerate(sentences) if \
                 i % sentences_per_doc == 0]
    batches = ['\n\n'.join(sentences[i:s]) for i, s in ranges]
    return batches

sentences = clean_texto(texto)
batches = create_batches(sentences, sentences_per_doc=16)
print('{} sentences found'.format(len(sentences)))
print('{} batches created'.format(len(batches)))

130 sentences found
9 batches created


Veamos las primeras oraciones del texto:

In [0]:
for s in sentences[:5]:
    print(s)

By this art you may contemplate the variations of the 23 letters...
The Anatomy of Melancholy, part 2, sect.
II, mem.
IV The universe (which others call the Library) is composed of an indefinite and perhaps infinite number of hexagonal galleries, with vast air shafts between, surrounded by very low railings.
From any of the hexagons one can see, interminably, the upper and lower floors.


### Dependencies parsing

*HOLA*

In [0]:
nlp = stanza.Pipeline('en', verbose=False)

*TAL CONVIENE USAR SENTENCES POR QUE DESPUES ES DIFICIL TRACKEAR LAS ORACIOENES ORIGINALES PORQUE NO HACE LOS CORTES EXACTAMENTE EN \N\N CUANDO SE USAN BATCHES*

In [0]:
def parse_sentences(batches):
    """Parse batched sentences 
    Returns: a list of docs, each one cointaining multiple parsed sentences
    """
    docs = list()
    for doc in batches:
        docs.append(nlp(doc))
    # ponemos los docs en una sola lista
    docs = list(docs)
    return docs
# docs = parse_sentences(batches)
docs = parse_sentences(sentences)

In [0]:
lengths = [len(doc.sentences) for doc in docs]
for i in lengths:
    print(i)
# No siempre da 1!!! (pero se puede corregir el idx de abajo creo)

In [0]:
def find_dependencies(docs):
    """Find amod dependencies in parsed docs
    Returns: list of dependencies with (sentence_idx, head, dependent)
    """
    deps = list()
    sent_idx = 0
    for doc in docs:
        for sent in doc.sentences:
            id2word = {word.id: word.text for word in sent.words}
            deps += [(sent_idx, id2word[str(word.head)], word.text) \
                    for word in sent.words if word.deprel=='amod']
        sent_idx += 1
    return deps
deps = find_dependencies(docs)

In [0]:
print(deps[len(deps)-1])
sentences[129]
# funciona el rastreo de oraciones!! :)

(129, 'page', 'inconceivable')


'The handling of this silky vade mecum would not be convenient: each apparent page would unfold into other analogous ones; the inconceivable middle page would have no reverse.'

## Word embeddings